In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pprint import pprint
%matplotlib inline
%load_ext autoreload
%timeit
%autoreload 2

pd.set_option('display.max_columns', 500)

In [2]:
def load_data():
    path = "data/processed_synopsis/"
    train_X = pd.read_hdf(path + "train_X.hdf5", "table")
    train_Y = pd.read_hdf(path + "train_Y.hdf5", "table")
    test_X = pd.read_hdf(path + "test_X.hdf5", "table")
    test_Y = pd.read_hdf(path + "test_Y.hdf5", "table")
    return train_X, train_Y, test_X, test_Y

In [3]:
train_X, train_Y, test_X, test_Y = load_data()

## TODO:
- clean reviews
- consider using: svm, trees, logistic regression
- read up on metrics: https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html

In [4]:
from sklearn import svm
from sklearn.model_selection import cross_val_score

In [5]:
train_X.head()

,mal_id,title_english,dropped,favorites,plan_to_watch,watching,season_Fall,season_Spring,season_Summer,season_Winter,year_2005,year_2006,year_2007,year_2008,year_2009,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018,is_Very_Popular,is_Popular,is_Unpopular,is_Very_Unpopular,has_Prequel,has_Sequel,is_Light novel,is_Manga,is_Original,is_Visual novel,is_Action,is_Adventure,is_Comedy,is_Drama,is_Ecchi,is_Fantasy,is_Harem,is_Josei,is_Mystery,is_Romance,is_School,is_Sci-Fi,is_Seinen,is_Shoujo,is_Shounen,is_Slice of Life,is_Supernatural,sv_0,sv_1,sv_2,sv_3,sv_4,sv_5,sv_6,sv_7,sv_8,sv_9,sv_10,sv_11,sv_12,sv_13,sv_14,sv_15,sv_16,sv_17,sv_18,sv_19,sv_20,sv_21,sv_22,sv_23,sv_24,sv_25,sv_26,sv_27,sv_28,sv_29,sv_30,sv_31,sv_32,sv_33,sv_34,sv_35,sv_36,sv_37,sv_38,sv_39,sv_40,sv_41,sv_42,sv_43,sv_44,sv_45,sv_46,sv_47,sv_48,sv_49,sv_50,sv_51,sv_52,sv_53,sv_54,sv_55,sv_56,sv_57,sv_58,sv_59,sv_60,sv_61,sv_62,sv_63,sv_64,sv_65,sv_66,sv_67,sv_68,sv_69,sv_70,sv_71,sv_72,sv_73,sv_74,sv_75,sv_76,sv_77,sv_78,sv_79,sv_80,sv_81,sv_82,sv_83,sv_84,sv_85,sv_86,sv_87,sv_88,sv_89,sv_90,sv_91,sv_92,sv_93,sv_94,sv_95,sv_96,sv_97,sv_98,sv_99,sv_100,sv_101,sv_102,sv_103,sv_104,sv_105,sv_106,sv_107,sv_108,sv_109,sv_110,sv_111,sv_112,sv_113,sv_114,sv_115,sv_116,sv_117,sv_118,sv_119,sv_120,sv_121,sv_122,sv_123,sv_124,sv_125,sv_126,sv_127,sv_128,sv_129,sv_130,sv_131,sv_132,sv_133,sv_134,sv_135,sv_136,sv_137,sv_138,sv_139,sv_140,sv_141,sv_142,sv_143,sv_144,sv_145,sv_146,sv_147,sv_148,sv_149,sv_150,sv_151,sv_152,sv_153,sv_154,sv_155,sv_156,sv_157,sv_158,sv_159,sv_160,sv_161,sv_162,sv_163,sv_164,sv_165,sv_166,sv_167,sv_168,sv_169,sv_170,sv_171,sv_172,sv_173,sv_174,sv_175,sv_176,sv_177,sv_178,sv_179,sv_180,sv_181,sv_182,sv_183,sv_184,sv_185,sv_186,sv_187,sv_188,sv_189,sv_190,sv_191,sv_192,sv_193,sv_194,sv_195,sv_196,sv_197,sv_198,sv_199,sv_200,sv_201,sv_202,sv_203,sv_204,sv_205,sv_206,sv_207,sv_208,sv_209,sv_210,sv_211,sv_212,sv_213,sv_214,sv_215,sv_216,sv_217,sv_218,sv_219,sv_220,sv_221,sv_222,sv_223,sv_224,sv_225,sv_226,sv_227,sv_228,sv_229,sv_230,sv_231,sv_232,sv_233,sv_234,sv_235,sv_236,sv_237,sv_238,sv_239,sv_240,sv_241,sv_242,sv_243,sv_244,sv_245,sv_246,sv_247,sv_248,sv_249,sv_250,sv_251,sv_252,sv_253,sv_254,sv_255,sv_256,sv_257,sv_258,sv_259,sv_260,sv_261,sv_262,sv_263,sv_264,sv_265,sv_266,sv_267,sv_268,sv_269,sv_270,sv_271,sv_272,sv_273,sv_274,sv_275,sv_276,sv_277,sv_278,sv_279,sv_280,sv_281,sv_282,sv_283,sv_284,sv_285,sv_286,sv_287,sv_288,sv_289,sv_290,sv_291,sv_292,sv_293,sv_294,sv_295,sv_296,sv_297,sv_298,sv_299
790,5630,Eden of The East,0.857602,-1.569807,-0.365276,1.066136,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,1,1,0,1,0,0,0,0,0,-0.049758,0.070097,-0.013160,0.029296,-0.044146,-0.044970,-0.055462,-0.023865,-0.003372,2.051685,-0.020228,-0.062936,-0.058681,-0.017387,-0.049538,0.010696,-0.042397,0.617382,-0.002686,0.032320,-0.018845,-0.044057,-0.024150,0.020822,0.002409,0.045144,0.041674,-0.032702,-0.009256,0.032120,-0.046420,-0.033880,-0.108927,0.011874,0.041124,0.044727,0.086923,0.000375,0.020178,0.002146,0.012383,-0.082690,0.125003,0.057947,-0.035477,0.053562,-0.066956,0.008789,0.026393,-0.055111,-0.094816,-0.000699,-0.090763,-0.048135,-0.054672,0.042932,0.019006,-0.103644,0.069593,-0.074405,-0.002536,-0.075918,0.020236,-0.134388,0.036675,-0.059457,-0.056521,0.088512,0.037646,0.011802,0.010475,-0.021126,0.013673,-0.025007,0.023995,-0.058334,0.038847,-0.115593,-0.011587,0.172999,0.019613,0.038611,-0.122097,-0.004023,0.039357,-0.131059,0.106670,-0.182603,0.019326,0.028611,-0.086143,0.122741,-0.144791,-0.003019,-0.014749,-0.042550,0.010953,-0.084592,-0.058311,0.043956,-0.026148,0.030212,0.020539,-0.070896,0.073315,-0.964839,-0.101054,0.114312,0.027857,0.059342,0.116553,0.025719,-0.004239,-0.077048,-0.053274,0.072610,0.050920,0.103574,-0.000879,-0.013368,-0.045377,-0.056739,0.009959,0.031102,0.121459,0.057266,-0.022258,-0.017304,-0.027942,0.038878,0.013859,-0.022585,0.033855,0.076539,0.006

In [6]:
train_Y.head()

,is_Poor,is_Below_Average,is_Average,is_Above_Average,is_Excellent
790,0,0,1,0,0
185,0,1,0,0,0
363,0,0,0,1,0
888,0,0,0,0,1
531,1,0,0,0,0


I made the mistake of thinking sklearn expects one hot class labels for multiclass classification. Now I have to convert the one hot labels back to categorical.

In [7]:
train_Y = train_Y.idxmax(axis=1)
test_Y = test_Y.idxmax(axis=1)

In [8]:
train_id = train_X[["mal_id", "title_english"]]
test_id = test_X[["mal_id", "title_english"]]

In [9]:
train_X.drop(labels=["mal_id", "title_english"], axis=1, inplace=True)
test_X.drop(labels=["mal_id", "title_english"], axis=1, inplace=True)

In [10]:
train_Y.value_counts()

is_Average          280
is_Poor             140
is_Below_Average    138
is_Above_Average     71
is_Excellent         57
dtype: int64

## Training SVM
Perform stratified 5-fold CV to tune C

In [26]:
svm_models = []
scores = []
C = np.logspace(start=-3, stop=3, base=10, num=20)

In [27]:
%%timeit
for c in np.nditer(C):
    svm_model = svm.SVC(C=c, kernel="rbf", gamma="scale", random_state=1)
    score = cross_val_score(svm_model, train_X, train_Y, cv=5)
    svm_models.append(svm_model)
    scores.append(score)

19.9 s ± 649 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
for score in scores:
     print("Accuracy: %0.2f (+/- %0.2f)" % (score.mean(), score.std()*2))

Accuracy: 0.41 (+/- 0.01)
Accuracy: 0.41 (+/- 0.01)
Accuracy: 0.41 (+/- 0.01)
Accuracy: 0.41 (+/- 0.01)
Accuracy: 0.41 (+/- 0.01)
Accuracy: 0.41 (+/- 0.01)
Accuracy: 0.46 (+/- 0.03)
Accuracy: 0.53 (+/- 0.02)
Accuracy: 0.57 (+/- 0.04)
Accuracy: 0.64 (+/- 0.02)
Accuracy: 0.67 (+/- 0.05)
Accuracy: 0.67 (+/- 0.05)
Accuracy: 0.65 (+/- 0.07)
Accuracy: 0.66 (+/- 0.10)
Accuracy: 0.65 (+/- 0.10)
Accuracy: 0.65 (+/- 0.10)
Accuracy: 0.65 (+/- 0.10)
Accuracy: 0.65 (+/- 0.10)
Accuracy: 0.65 (+/- 0.10)
Accuracy: 0.65 (+/- 0.10)
Accuracy: 0.41 (+/- 0.01)
Accuracy: 0.41 (+/- 0.01)
Accuracy: 0.41 (+/- 0.01)
Accuracy: 0.41 (+/- 0.01)
Accuracy: 0.41 (+/- 0.01)
Accuracy: 0.41 (+/- 0.01)
Accuracy: 0.46 (+/- 0.03)
Accuracy: 0.53 (+/- 0.02)
Accuracy: 0.57 (+/- 0.04)
Accuracy: 0.64 (+/- 0.02)
Accuracy: 0.67 (+/- 0.05)
Accuracy: 0.67 (+/- 0.05)
Accuracy: 0.65 (+/- 0.07)
Accuracy: 0.66 (+/- 0.10)
Accuracy: 0.65 (+/- 0.10)
Accuracy: 0.65 (+/- 0.10)
Accuracy: 0.65 (+/- 0.10)
Accuracy: 0.65 (+/- 0.10)
Accuracy: 0.

In [35]:
mean_acc = [score.mean() for score in scores]
np.argmax(mean_acc)

10

## Training DT